<a href="https://colab.research.google.com/github/gc2321/3235-Machine-Learning/blob/main/big5/Big5_personality_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
random.seed(42)

# Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import fileinput

# location of big5 personality scores data
filePath ="/content/gdrive/MyDrive/big5/data/big5_personality_scores/data-final.csv"

In [3]:
# Create dataframe
import pandas as pd
data = pd.read_csv(filePath, delimiter='\t')

In [4]:
data.shape

(1015341, 110)

In [5]:
data.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0


In [6]:
# get the first 50 columns (include only answers to all questions)
data = data.iloc[:, :50]

In [7]:
data.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,5.0,1.0,5.0,1.0,5.0,1.0,5.0,3.0,5.0,5.0


In [8]:
# clean data of missing values
data = data.dropna()

In [9]:
# Check for NaN values in the entire DataFrame
nan_values_overall = data.isna().sum().sum()

# Check for NaN values column-wise
nan_values_per_column = data.isna().sum()

# Display the total number of NaN values and NaN values per column
print("Total NaN values:", nan_values_overall)
print("\nNaN values per column:")
print(nan_values_per_column)

Total NaN values: 0

NaN values per column:
EXT1     0
EXT2     0
EXT3     0
EXT4     0
EXT5     0
EXT6     0
EXT7     0
EXT8     0
EXT9     0
EXT10    0
EST1     0
EST2     0
EST3     0
EST4     0
EST5     0
EST6     0
EST7     0
EST8     0
EST9     0
EST10    0
AGR1     0
AGR2     0
AGR3     0
AGR4     0
AGR5     0
AGR6     0
AGR7     0
AGR8     0
AGR9     0
AGR10    0
CSN1     0
CSN2     0
CSN3     0
CSN4     0
CSN5     0
CSN6     0
CSN7     0
CSN8     0
CSN9     0
CSN10    0
OPN1     0
OPN2     0
OPN3     0
OPN4     0
OPN5     0
OPN6     0
OPN7     0
OPN8     0
OPN9     0
OPN10    0
dtype: int64


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1013558 entries, 0 to 1015340
Data columns (total 50 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   EXT1    1013558 non-null  float64
 1   EXT2    1013558 non-null  float64
 2   EXT3    1013558 non-null  float64
 3   EXT4    1013558 non-null  float64
 4   EXT5    1013558 non-null  float64
 5   EXT6    1013558 non-null  float64
 6   EXT7    1013558 non-null  float64
 7   EXT8    1013558 non-null  float64
 8   EXT9    1013558 non-null  float64
 9   EXT10   1013558 non-null  float64
 10  EST1    1013558 non-null  float64
 11  EST2    1013558 non-null  float64
 12  EST3    1013558 non-null  float64
 13  EST4    1013558 non-null  float64
 14  EST5    1013558 non-null  float64
 15  EST6    1013558 non-null  float64
 16  EST7    1013558 non-null  float64
 17  EST8    1013558 non-null  float64
 18  EST9    1013558 non-null  float64
 19  EST10   1013558 non-null  float64
 20  AGR1    1013558 non-null

In [11]:
column_names = data.columns
X = data.values

In [12]:
column_names

Index(['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9',
       'EXT10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8',
       'EST9', 'EST10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7',
       'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6',
       'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5',
       'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10'],
      dtype='object')

In [13]:
X.shape

(1013558, 50)

In [14]:
# take a subset for finding getting silhouette_avgs
num_samples = 100000
random_indices = np.random.choice(X.shape[0], size=num_samples, replace=False)
sub_X = X[random_indices]

In [15]:
sub_X.shape

(100000, 50)

In [16]:
#try different K values for clustering and find the optimal K

# inertias_1 = []
# silhouette_avgs = []
# max_K = 10
# for k in np.arange(1,max_K+1):
#     kmeans = KMeans(n_clusters=k, random_state=0).fit(sub_X)
#     inertias_1 = inertias_1 + [kmeans.inertia_]
#     if k>1:
#         silhouette_avgs = silhouette_avgs + [silhouette_score(sub_X, kmeans.labels_)]

In [17]:
# plt.figure(figsize=(8, 3.5))
# plt.plot(range(1, 11), inertias_1, "bo-")
# plt.xlabel("$k$")
# plt.ylabel("Inertia")
# plt.grid()
# plt.show()

In [18]:
# plt.figure(figsize=(8, 3))

# k_range = range(2, 11)
# best_index = np.argmax(silhouette_avgs)
# best_k = k_range[best_index]
# best_score = silhouette_avgs[best_index]

# plt.plot(k_range, silhouette_avgs, "bo-")
# plt.xlabel("$k$")
# plt.ylabel("Silhouette score")
# plt.plot(best_k, best_score, "rs")
# plt.grid()
# plt.show()

In [19]:
# using 5 clusters using all of the data
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [20]:
from itertools import compress

cluster_0 = {}
cluster_1 = {}
cluster_2 = {}
cluster_3 = {}
cluster_4 = {}

def add_to_dict(dict, key, val):
  if key in dict:
    dict[key].append(val)
  else:
    dict[key] = [val]

for cluster_id in np.unique(kmeans.labels_):
  print("Cluster", cluster_id)

  in_cluster = kmeans.labels_ == cluster_id
  indices = np.where(in_cluster)[0]

  for index in indices:
    row = data.iloc[index]
    for column, val in row.items():
      if cluster_id == 0:
        add_to_dict(cluster_0, column, val)
      elif cluster_id == 1:
        add_to_dict(cluster_1, column, val)
      elif cluster_id == 2:
        add_to_dict(cluster_2, column, val)
      elif cluster_id == 3:
        add_to_dict(cluster_3, column, val)
      elif cluster_id == 4:
        add_to_dict(cluster_4, column, val)


Cluster 0
Cluster 1
Cluster 2
Cluster 3
Cluster 4


In [21]:
questions = {'EXT1': 'I am the life of the party.', 'EXT2': "I don't talk a lot.", 'EXT3': 'I feel comfortable around people.', 'EXT4': 'I keep in the background.', 'EXT5': 'I start conversations.', 'EXT6': 'I have little to say.', 'EXT7': 'I talk to a lot of different people at parties.', 'EXT8': "I don't like to draw attention to myself.", 'EXT9': "I don't mind being the center of attention.", 'EXT10': 'I am quiet around strangers.', 'EST1': 'I get stressed out easily.', 'EST2': 'I am relaxed most of the time.', 'EST3': 'I worry about things.', 'EST4': 'I seldom feel blue.', 'EST5': 'I am easily disturbed.', 'EST6': 'I get upset easily.', 'EST7': 'I change my mood a lot.', 'EST8': 'I have frequent mood swings.', 'EST9': 'I get irritated easily.', 'EST10': 'I often feel blue.', 'AGR1': 'I feel little concern for others.', 'AGR2': 'I am interested in people.', 'AGR3': 'I insult people.', 'AGR4': "I sympathize with others' feelings.", 'AGR5': "I am not interested in other people's problems.", 'AGR6': 'I have a soft heart.', 'AGR7': 'I am not really interested in others.', 'AGR8': 'I take time out for others.', 'AGR9': "I feel others' emotions.", 'AGR10': 'I make people feel at ease.', 'CSN1': 'I am always prepared.', 'CSN2': 'I leave my belongings around.', 'CSN3': 'I pay attention to details.', 'CSN4': 'I make a mess of things.', 'CSN5': 'I get chores done right away.', 'CSN6': 'I often forget to put things back in their proper place.', 'CSN7': 'I like order.', 'CSN8': 'I shirk my duties.', 'CSN9': 'I follow a schedule.', 'CSN10': 'I am exacting in my work.', 'OPN1': 'I have a rich vocabulary.', 'OPN2': 'I have difficulty understanding abstract ideas.', 'OPN3': 'I have a vivid imagination.', 'OPN4': 'I am not interested in abstract ideas.', 'OPN5': 'I have excellent ideas.', 'OPN6': 'I do not have a good imagination.', 'OPN7': 'I am quick to understand things.', 'OPN8': 'I use difficult words.', 'OPN9': 'I spend time reflecting on things.', 'OPN10': 'I am full of ideas.'}


In [22]:
# cluster 0
for key, value in cluster_0.items():
  avg = sum(value)/len(value)
  if avg > 4 or avg < 2:
    print(questions[key], "->", round(avg,1))

I am the life of the party. -> 1.8
I keep in the background. -> 4.1
I talk to a lot of different people at parties. -> 1.7
I don't like to draw attention to myself. -> 4.1
I am quiet around strangers. -> 4.5
I get stressed out easily. -> 4.2
I worry about things. -> 4.5
I get irritated easily. -> 4.0
I have a vivid imagination. -> 4.1
I spend time reflecting on things. -> 4.3


In [23]:
# cluster 1
for key, value in cluster_1.items():
  avg = sum(value)/len(value)
  if avg > 4 or avg < 2:
    print(questions[key], "->", round(avg,1))

I don't talk a lot. -> 1.9
I feel comfortable around people. -> 4.3
I start conversations. -> 4.3
I have little to say. -> 1.7
I get upset easily. -> 1.9
I have frequent mood swings. -> 1.7
I often feel blue. -> 1.7
I feel little concern for others. -> 1.9
I am interested in people. -> 4.4
I insult people. -> 1.8
I sympathize with others' feelings. -> 4.2
I am not interested in other people's problems. -> 1.9
I am not really interested in others. -> 1.6
I take time out for others. -> 4.1
I feel others' emotions. -> 4.1
I make people feel at ease. -> 4.2
I pay attention to details. -> 4.2
I make a mess of things. -> 1.9
I shirk my duties. -> 1.9
I have difficulty understanding abstract ideas. -> 1.8
I have a vivid imagination. -> 4.1
I am not interested in abstract ideas. -> 1.8
I have excellent ideas. -> 4.2
I do not have a good imagination. -> 1.7
I am quick to understand things. -> 4.3
I spend time reflecting on things. -> 4.1
I am full of ideas. -> 4.3


In [24]:
# cluster 2
for key, value in cluster_2.items():
  avg = sum(value)/len(value)
  if avg > 4 or avg < 2:
    print(questions[key], "->", round(avg,1))

I don't talk a lot. -> 1.9
I start conversations. -> 4.0
I have little to say. -> 1.8
I worry about things. -> 4.2
I am interested in people. -> 4.3
I sympathize with others' feelings. -> 4.2
I am not really interested in others. -> 1.9
I feel others' emotions. -> 4.1
I have a vivid imagination. -> 4.2
I am not interested in abstract ideas. -> 2.0
I do not have a good imagination. -> 1.8
I spend time reflecting on things. -> 4.2
I am full of ideas. -> 4.2


In [25]:
# cluster 3
for key, value in cluster_3.items():
  avg = sum(value)/len(value)
  if avg > 4 or avg < 2:
    print(questions[key], "->", round(avg,1))

I don't like to draw attention to myself. -> 4.0
I am quiet around strangers. -> 4.1
I worry about things. -> 4.2
I feel little concern for others. -> 1.9
I insult people. -> 1.7
I sympathize with others' feelings. -> 4.3
I am not interested in other people's problems. -> 2.0
I have a soft heart. -> 4.1
I feel others' emotions. -> 4.2
I pay attention to details. -> 4.3
I like order. -> 4.2
I do not have a good imagination. -> 2.0
I am quick to understand things. -> 4.0
I spend time reflecting on things. -> 4.3


In [26]:
# cluster 4
for key, value in cluster_4.items():
  avg = sum(value)/len(value)
  if avg > 4 or avg < 2:
    print(questions[key], "->", round(avg,1))

I have frequent mood swings. -> 1.9
I have difficulty understanding abstract ideas. -> 2.0


In [27]:
# use two clusters on all of the ata

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [31]:
cluster_0 = {}
cluster_1 = {}
cluster_0_count = 0
cluster_1_count = 0

for cluster_id in np.unique(kmeans.labels_):
  print("Cluster", cluster_id)

  in_cluster = kmeans.labels_ == cluster_id
  indices = np.where(in_cluster)[0]

  if cluster_id == 0:
    cluster_0_count = len(indices)
  elif cluster_id == 1:
    cluster_1_count = len(indices)

  for index in indices:
    row = data.iloc[index]
    for column, val in row.items():
      if cluster_id == 0:
        add_to_dict(cluster_0, column, val)
      elif cluster_id == 1:
        add_to_dict(cluster_1, column, val)


Cluster 0
Cluster 1


In [39]:
# cluster 0
for key, value in cluster_0.items():
  avg = sum(value)/len(value)
  if avg > 3.8 or avg < 2.3:
    print(key, questions[key], "->", round(avg,1))

print('...........')
print(f"this cluster has {cluster_0_count} entries")

EXT2 I don't talk a lot. -> 2.1
EXT3 I feel comfortable around people. -> 4.0
EXT5 I start conversations. -> 4.0
EXT6 I have little to say. -> 1.9
EST8 I have frequent mood swings. -> 2.2
EST10 I often feel blue. -> 2.1
AGR1 I feel little concern for others. -> 2.0
AGR2 I am interested in people. -> 4.2
AGR3 I insult people. -> 2.1
AGR4 I sympathize with others' feelings. -> 4.1
AGR5 I am not interested in other people's problems. -> 2.0
AGR7 I am not really interested in others. -> 1.8
AGR8 I take time out for others. -> 3.9
AGR9 I feel others' emotions. -> 4.0
AGR10 I make people feel at ease. -> 4.0
CSN3 I pay attention to details. -> 4.0
CSN4 I make a mess of things. -> 2.2
CSN8 I shirk my duties. -> 2.2
OPN2 I have difficulty understanding abstract ideas. -> 1.9
OPN3 I have a vivid imagination. -> 4.0
OPN4 I am not interested in abstract ideas. -> 1.9
OPN5 I have excellent ideas. -> 4.0
OPN6 I do not have a good imagination. -> 1.7
OPN7 I am quick to understand things. -> 4.1
OPN9

In [40]:
# cluster 1
for key, value in cluster_1.items():
  avg = sum(value)/len(value)
  if avg > 3.8 or avg < 2.3:
    print(key, questions[key], "->", round(avg,1))

print('...........')
print(f"this cluster has {cluster_1_count} entries")

EXT1 I am the life of the party. -> 2.1
EXT7 I talk to a lot of different people at parties. -> 2.1
EXT8 I don't like to draw attention to myself. -> 3.9
EXT10 I am quiet around strangers. -> 4.2
EST3 I worry about things. -> 4.2
CSN3 I pay attention to details. -> 3.9
OPN2 I have difficulty understanding abstract ideas. -> 2.3
OPN3 I have a vivid imagination. -> 4.0
OPN4 I am not interested in abstract ideas. -> 2.1
OPN6 I do not have a good imagination. -> 2.1
OPN7 I am quick to understand things. -> 3.8
OPN9 I spend time reflecting on things. -> 4.2
...........
this cluster has 515581 entries


In [34]:
print(f'the ratio of cluster_0 and cluster_1 is {round(cluster_0_count/cluster_1_count, 2)}')

the ratio of cluster_0 and cluster_1 is 0.97


In [45]:
# get difference of two groups
diff_dict = {}

for key in cluster_0.keys():
  avg_0 = sum(cluster_0[key]) / len(cluster_0[key])
  avg_1 = sum(cluster_1[key]) / len(cluster_1[key])

  diff = abs((avg_0 - avg_1)/2)
  diff_dict[key]=diff

In [53]:
sorted_diff_dict = dict(sorted(diff_dict.items(), key=lambda item: item[1]*-1))
for key, value in sorted_diff_dict.items():
  avg_0 = sum(cluster_0[key]) / len(cluster_0[key])
  avg_1 = sum(cluster_1[key]) / len(cluster_1[key])
  print(key, questions[key], round(value,2), 'cluster_0 scores ', round(avg_0, 1), ' cluster_1 scores ', round(avg_1,1))

EXT7 I talk to a lot of different people at parties. 0.73 cluster_0 scores  3.5  cluster_1 scores  2.1
EXT3 I feel comfortable around people. 0.69 cluster_0 scores  4.0  cluster_1 scores  2.6
EXT10 I am quiet around strangers. 0.69 cluster_0 scores  2.9  cluster_1 scores  4.2
EXT5 I start conversations. 0.67 cluster_0 scores  4.0  cluster_1 scores  2.6
EXT4 I keep in the background. 0.65 cluster_0 scores  2.5  cluster_1 scores  3.8
EST10 I often feel blue. 0.62 cluster_0 scores  2.1  cluster_1 scores  3.4
EXT2 I don't talk a lot. 0.62 cluster_0 scores  2.1  cluster_1 scores  3.4
EXT1 I am the life of the party. 0.58 cluster_0 scores  3.2  cluster_1 scores  2.1
EXT6 I have little to say. 0.54 cluster_0 scores  1.9  cluster_1 scores  2.9
EXT9 I don't mind being the center of attention. 0.54 cluster_0 scores  3.5  cluster_1 scores  2.4
EST8 I have frequent mood swings. 0.52 cluster_0 scores  2.2  cluster_1 scores  3.2
EST1 I get stressed out easily. 0.51 cluster_0 scores  2.8  cluster_1 s

In [54]:
# difference between cluster_0 and cluster_1 are largely due to difference in EXT question, cluster_0 scores higher in Extraversion, cluster_1 score lower in Extraversion, and higher in some Neuroticism questions
# print all questions with difference  > 0.4

for key, value in sorted_diff_dict.items():
  if value > 0.4:
    print(key, questions[key], round(value,2))

EXT7 I talk to a lot of different people at parties. 0.73
EXT3 I feel comfortable around people. 0.69
EXT10 I am quiet around strangers. 0.69
EXT5 I start conversations. 0.67
EXT4 I keep in the background. 0.65
EST10 I often feel blue. 0.62
EXT2 I don't talk a lot. 0.62
EXT1 I am the life of the party. 0.58
EXT6 I have little to say. 0.54
EXT9 I don't mind being the center of attention. 0.54
EST8 I have frequent mood swings. 0.52
EST1 I get stressed out easily. 0.51
EST9 I get irritated easily. 0.5
EST6 I get upset easily. 0.5
EXT8 I don't like to draw attention to myself. 0.47
EST7 I change my mood a lot. 0.46
AGR7 I am not really interested in others. 0.42


In [55]:
for key, value in sorted_diff_dict.items():
  if value > 0.4:
    print(key, round(value,2))

EXT7 0.73
EXT3 0.69
EXT10 0.69
EXT5 0.67
EXT4 0.65
EST10 0.62
EXT2 0.62
EXT1 0.58
EXT6 0.54
EXT9 0.54
EST8 0.52
EST1 0.51
EST9 0.5
EST6 0.5
EXT8 0.47
EST7 0.46
AGR7 0.42


In [56]:
# establish a score for separating cluster_0 and cluster_1

cluster_dict = {'EXT7': 0.73, 'EXT3': 0.69, 'EXT10': 0.69, 'EXT5': 0.67, 'EXT4': 0.65, 'EST10': 0.62, 'EXT2': 0.62, 'EXT1': 0.58, 'EXT6': 0.54, 'EXT9': 0.54, 'EST8': 0.52, 'EST1': 0.51, 'EST9': 0.5, 'EST6': 0.5, 'EXT8': 0.47, 'EST7': 0.46, 'AGR7': 0.42}